<a href="https://colab.research.google.com/github/kfuentestorres/Master-BI-BDA---Homeworks/blob/master/Review_Bombing_TLOUII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installo le librerie

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas
import json
import pprint

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Scarico la lista dei primi 1000 giochi secondo lo score di Metacritic



In [ ]:
def parse_review(review):
  try:
    review_id = review.find_elements_by_css_selector("input")[0].get_attribute("id")
    title = review.find_elements_by_css_selector("a h3")[0].text
    url = review.find_elements_by_css_selector("a.title")[0].get_attribute("href")
  except:
    pass
  return {'review_id': review_id,
          'url': url,
          'title': title}


In [ ]:
import time
detail_reviews = []
for num in tqdm(range(0,10)):
  time.sleep(1)
  wd.get(f"https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page={num}")
  list_reviews = wd.find_elements_by_css_selector("td.clamp-summary-wrap")
  for review in list_reviews:
    detail_reviews.append(parse_review(review))

print(len(detail_reviews))

In [ ]:
import pandas as pd
ds_detail_reviews = pd.DataFrame(detail_reviews)
ds_detail_reviews.set_index("review_id")
ds_detail_reviews.head()

In [ ]:
ds_detail_reviews.info()

In [ ]:
ds_detail_reviews.to_csv('ds_reviews.csv')

# Estraggo le informazioni dei primi 1000 giochi

In [ ]:
# open csv file
import pandas as pd
ds_detail_reviews = pd.read_csv("ds_reviews.csv", index_col="review_id")
ds_detail_reviews.head()

In [ ]:
detail_score = []
for review_id, review in tqdm(ds_detail_reviews.iterrows(),total=ds_detail_reviews.shape[0]):
  time.sleep(1)
  url = review["url"]
  wd.get(url)
  try:
    title = wd.find_elements_by_css_selector("a h1")[0].text
    platform = wd.find_elements_by_css_selector("li.summary_detail.publisher > span.data > a")[0].text
    publisher = wd.find_elements_by_css_selector("li.publisher span.data")[0].text
    developer = wd.find_elements_by_css_selector("li.developer span.data")[0].text
    genre = wd.find_elements_by_css_selector("li.product_genre")[0].text
    players = wd.find_elements_by_css_selector("li.product_players span.data")[0].text
    release = wd.find_elements_by_css_selector("li.release_data span.data")[0].text
    score_meta = wd.find_elements_by_css_selector("div.game span")[0].text
    score_user = wd.find_elements_by_css_selector("div.details.side_details > div:nth-child(1) > div > a > div")[0].text
  except:
    pass
  detail_score.append ({'review_id': review_id,
                      'title': title,
                      'platform': platform,
                      'publisher': publisher,
                      'developer': developer,
                      'genre': genre,
                      'players': players,
                      'release': release,
                      'score_meta': score_meta,
                      'score_user': score_user})

In [ ]:
ds_detail_score = pd.DataFrame(detail_score)
ds_detail_score.set_index("review_id")
ds_detail_score.head()

In [ ]:
ds_detail_score.to_csv('ds_reviews_score.csv')

# Estraggo le informazioni delle prime 1000 recensioni di "The Last of Us Part II"

In [ ]:
def parse_review_user(review):
  try:
    review_id = review.get_attribute("id")
    username = review.find_elements_by_css_selector("div.name")[0].text
    score = review.find_elements_by_css_selector("div.metascore_w")[0].text
    release = review.find_elements_by_css_selector("div.date")[0].text
    try:
      review.find_element_by_css_selector("span.toggle_expand").click()
      body = review.find_elements_by_css_selector("div.review_body")[0].text
    except:
      body = review.find_elements_by_css_selector("div.review_body")[0].text
  except:
    pass
  return {'review_id': review_id,
          'username': username,
          'score': score,
          'release': release,
          'body': body}

In [ ]:
import time
detail_reviews_user = []
for num in tqdm(range(0,10)):
  time.sleep(1)
  wd.get(f"https://www.metacritic.com/game/playstation-4/the-last-of-us-part-ii/user-reviews?page={num}")
  list_reviews = wd.find_elements_by_css_selector("li.user_review")
  for review in list_reviews:
    detail_reviews_user.append(parse_review_user(review))

print(len(detail_reviews))

In [ ]:
ds_detail_reviews_user = pd.DataFrame(detail_reviews_user)
ds_detail_reviews_user.set_index("review_id")
ds_detail_reviews_user.head()

In [ ]:
ds_detail_reviews_user.info()

In [ ]:
ds_detail_reviews_user.to_csv('ds_reviews_user.csv')